# Rock paper scissor 

Use either/both of the following websites to play the game. 
- http://www.essentially.net/rsp/play.jsp

- https://www.afiniti.com/corporate/rock-paper-scissors

A truly random game of rock-paper-scissors would result in a statistical tie with each player winning, tying and losing one-third of the time. However, people are not truly random and thus can be studied and analyzed. While this computer won’t win all rounds, over time it can exploit a person’s tendencies and patterns to gain an advantage over its opponent.

Now play the game of the rock-paper-scissors against the computer at least 100 times while you record the outcome (your moves & the computer’s move) at each time into a text ﬁle. Record the score at the bottom of the ﬁle, i.e. your number of wins, losses and ties out of N tries (N ≥ 100). Now use this ﬁle to come up with some algorithm that takes as input the history to how the game has been played and estimate your best move for the next round. Put this algorithm into a simple program (use python 3.x) and play the computer again at least the same number of times you played before, but this time using your program. Record the score (hopefully there is an improvement in your score!!). Hint: the more you play the better you may be at guessing what the computer is up to. Also, note that we haven’t covered any ML techniques yet in the class yet, so this is just a warm up problem, so just do the best you can. You can also develop your playing system using data from one website and see if you can use it to improve your performance on the other website (transfer learning and generalization), though this is optional. 

After you implemented your algorithm and model, you may use it manually with the computer by type in the computer response at each round to your program, or you can write a script to interact with the webpage. But include at least two text files: history before using your algorithm, history after using your algorithm. 


In [1]:
# Data extraction here: extract the data from the txt file to python input
# your input can be of any format, one suggestion is using integers 0,1,2 for rock paper scissor

from google.colab import files
import random

In [2]:
#Define your algorithm here or define your own functions and procedure
# def rock_paper_scissor(history=None):
#     # define your function here
#     pass


## This simulation is for training, and it is just based on truly random matches ##

# rock = 0 , paper = 1 , scissor = 2
key = ["rock", "paper", "scissor"]

table = [["tie", "lose", "win"],
         ["win", "tie", "lose"],
         ["lose", "win", "tie"]]

# perf = [win_cnt, lose_cnt, tie_cnt]

def random_match(performance):
  me = random.randint(0,2)
  comp = random.randint(0,2)
  result = table[me][comp]
  if result == "win":
    performance[0] += 1
  elif result == "lose":
    performance[1] += 1
  else:
    performance[2] += 1
  return str(me) + " " + str(comp) + " " + result

def play_game(trial, matches):
  performance = [0, 0, 0]
  path = "simulation_" + str(trial) + ".txt"
  file = open(path, 'x')
  file.write("me | computer | outcome")
  for i in range(matches):
    throw = random_match(performance)
    file.write("\n" + throw)
  file.write("\n" + str(performance))
  file.close()
  return performance

# print(str(round(num, 2)) + "%")
display = ["win : ", "lose: ", "tie : "]

def run_simulation(test_no, trials):
  perf = play_game(test_no, trials)
  print("Round no." + str(test_no))
  for i in range(len(perf)):
    num = perf[i] / trials
    rate = round((num * 100), 2)
    #printing to console
    print(display[i] + str(rate) + "%")
  #more printing to console
  if perf[0] > perf[1]:
    print("YOU won!")
  elif perf[1] > perf[0]:
    print("Computer won!")
  else:
    print("very unlikely...")

# setting r-c-s training siumlation numbers
number_of_tests = 10
trials = 1000

print("\"You\" are playing against the computer for " + str(number_of_tests) + " rounds.")
print("Number of games per round: " + str(trials) + "\n")
for i in range(number_of_tests):
  #print("game #" + str(i+1))
  run_simulation(i, trials)
  print()

"You" are playing against the computer for 10 rounds.
Number of games per round: 1000

Round no.0
win : 34.3%
lose: 32.4%
tie : 33.3%
YOU won!

Round no.1
win : 33.6%
lose: 32.9%
tie : 33.5%
YOU won!

Round no.2
win : 34.1%
lose: 33.6%
tie : 32.3%
YOU won!

Round no.3
win : 33.2%
lose: 33.4%
tie : 33.4%
Computer won!

Round no.4
win : 34.5%
lose: 30.3%
tie : 35.2%
YOU won!

Round no.5
win : 33.4%
lose: 33.9%
tie : 32.7%
Computer won!

Round no.6
win : 31.0%
lose: 32.9%
tie : 36.1%
Computer won!

Round no.7
win : 37.1%
lose: 29.6%
tie : 33.3%
YOU won!

Round no.8
win : 32.7%
lose: 35.5%
tie : 31.8%
Computer won!

Round no.9
win : 31.9%
lose: 31.8%
tie : 36.3%
YOU won!



# New Section

In [3]:
# Implementig the AI to predict a match

read_path = "simulation_"

def train_ai(num_simulations, path):
  lines = []
  # history = { player_last_three_moves : {ai_next_move : ai_wins_for_strategy}}
  history = {'000' : {0: 0, 1: 0, 2: 0}}
  # comp_wins = [cnt_rock, cnt_paper, cnt_scissor]
  comp_wins = [0, 0, 0]

  # feeding all the training simulations into the network
  for i in range(num_simulations):
    current_file = open(read_path + str(i) + ".txt", 'r')
    file_lines = current_file.readlines()
    file_lines = file_lines[1:-1]
    for line in range(len(file_lines)): 
      file_lines[line] = file_lines[line][:-1]
    lines += file_lines

  my_cache = ''
  for match in range(len(lines)):
    game = lines[match].split()
    comp_move = int(game[1])
    if game[2] == 'lose':
      comp_wins[comp_move] += 1
    if match > 2: 
      comp_move = int(game[1])

      #print(str(my_cache) + " " + str(comp_move))
      if game[2] == 'lose': #this means computer won
        if my_cache not in history:
          history.update({my_cache : {comp_move : 1}})
        else:
          if comp_move not in history[my_cache]:
            history[my_cache].update({comp_move: 1})
          else:
            history[my_cache][comp_move] += 1
      
      my_cache = my_cache[1:]
      
    my_cache += game[0]

  return (comp_wins, history)


# Calling the AI training based on generated simulation data
# Could be more automated
(breakdown, strategy) = train_ai(number_of_tests, read_path)
print("Winnning rate: " + str(round((sum(breakdown) / (number_of_tests * trials) * 100), 2)) + "%\n")
print("Breakdown - [#wins_rock, #wins_paper, #wins_scissors]")
print(breakdown)
print()
print("Strategy dictionary: ")
print(strategy)
# max102 = max(strategy['102'], key=strategy['102'].get)
# print(max102)
# max212 = max(strategy['212'], key=strategy['212'].get)
# print(max212)
# max110 = max(strategy['110'], key=strategy['110'].get)
# print(max110)


# Really simple AI brain
def make_AI_move(previous_moves, cached_moves, strategy, breakdown):
  if len(previous_moves) == 3:
    #if not (previous_moves == cached_moves): 
    if previous_moves in strategy:
      return max(strategy[previous_moves], key=strategy[previous_moves].get)
  else:
    return breakdown.index(max(breakdown))

Winnning rate: 32.63%

Breakdown - [#wins_rock, #wins_paper, #wins_scissors]
[1120, 1059, 1084]

Strategy dictionary: 
{'000': {0: 46, 1: 41, 2: 45}, '012': {2: 45, 1: 47, 0: 52}, '112': {0: 34, 2: 38, 1: 30}, '122': {0: 51, 1: 42, 2: 36}, '011': {1: 43, 0: 37, 2: 32}, '022': {2: 38, 1: 47, 0: 35}, '212': {1: 45, 0: 43, 2: 43}, '220': {0: 55, 1: 37, 2: 42}, '221': {0: 40, 1: 46, 2: 47}, '111': {2: 41, 1: 39, 0: 37}, '020': {0: 43, 1: 34, 2: 33}, '201': {0: 40, 1: 34, 2: 41}, '120': {2: 34, 1: 51, 0: 39}, '222': {1: 36, 0: 44, 2: 39}, '200': {1: 40, 2: 34, 0: 33}, '001': {2: 56, 0: 45, 1: 27}, '211': {0: 23, 2: 36, 1: 41}, '002': {0: 37, 2: 39, 1: 32}, '102': {0: 47, 2: 40, 1: 32}, '010': {2: 53, 0: 44, 1: 35}, '101': {1: 37, 2: 37, 0: 46}, '110': {1: 35, 2: 45, 0: 35}, '100': {1: 49, 2: 36, 0: 38}, '202': {1: 46, 2: 40, 0: 48}, '021': {1: 31, 2: 35, 0: 37}, '210': {2: 37, 1: 35, 0: 47}, '121': {1: 46, 2: 41, 0: 44}}


In [4]:
 # Playing the game against the AI 

# NOTE: I did a shortcut to have trained the AI based on gameplays with random 
# computer matches, therefore the training set does not reflect my play style...
# Hopefully, the AI will still be able to beat me! 

# history(for the AI) = [win_cnt_rock, win_cnt_scissor, win_cnt ]


def play_game(debug=True, auto=False, repeat=1000):

  on_repeat = repeat # 1000 test simulations in auto case
  (breakdown, strategy) = train_ai(number_of_tests, read_path)
  og_b = breakdown.copy()
  cached_moves = ''
  cached_moves_copy = ''
  count = 0
  stats = [0, 0, 0]
  if debug: 
    print("Rock = 0 | Paper = 1 | Scissors = 2 | Enter 'q' to quit.")
  while on_repeat > 0:
    count += 1
    if auto:
      on_repeat -= 1
    if debug:
      print()
      print("Round " + str(count))
    if auto:
      player_input = str(random.randint(0,2))
    else:
      player_input = input("Please input: ")
      acceptable_inputs = ['0', '1', '2']
      if player_input == 'q':
        break
      elif player_input not in acceptable_inputs:
        print("Invalid input")
        break
    #recording move
    cached_moves += player_input
    if len(cached_moves) > 3:
      cached_moves = cached_moves[1:]
    move = int(player_input)

    ai_move = make_AI_move(cached_moves, cached_moves_copy, strategy, breakdown)
    cached_moves_copy = cached_moves
    (breakdown, strategy) = update_game(int(player_input), ai_move, cached_moves, breakdown, stats, strategy, debug=debug)
    
    if debug:
      print("(won-" + str(stats[0]) + " lost-" + str(stats[1]) + " tied-" + str(stats[2]) + ")")
  
  if debug:
    print()
  print("Over " + str(count) + " games... ")
  print("won " + str(stats[0]) + " times, ")
  print("lost " + str(stats[1]) + " times, ")
  print("and tied " + str(stats[2]) + " times. ")
  print("Winning rate = " + str(round((stats[0] / count * 100), 2)) + "%")


def update_game(player_input, ai_input, moves, breakdown, stats, strategy, debug=False):
  result = table[ai_input][player_input]
  if debug:
    print("AI [" + key[ai_input] + "] vs your [" + key[player_input] + 
              "] >>> " + result)
  if result == 'win':
    breakdown[ai_input] += 1
    stats[0] += 1
  elif result == 'lose':
    stats[1] += 1
  elif result == 'tie':
    stats[2] += 1

    strategy = update_strategy(ai_input, moves, strategy, True)


  return (breakdown, strategy)

def update_strategy(ai_input, moves, strategy, winlose):
  if moves in strategy:
    if ai_input in strategy[moves]:
      if winlose == True:
        strategy[moves][ai_input] += 1
      else:
        strategy[moves][ai_input] -= 1
    else:
      if winlose == True:
        strategy[moves].update({ai_input : 1})
      else:
        strategy[moves].update({ai_input : -1})
  else:
    if winlose == True:
      strategy.update({moves: {ai_input: 1}})
    else:
      strategy.update({moves: {ai_input: -1}})
  return strategy
  

# Assuming that you called run_simulation() and train_ai() already

print("My AI performance: ")
print()
#play_game()


print("Test #1")
play_game(debug=False, auto = True, repeat = 10000)

print("\nTest #2")
play_game(debug=False, auto = True, repeat = 20000)

print("\nTest #3")
play_game(debug=False, auto = True, repeat = 30000)

print("\nTest #4")
play_game(debug=False, auto = True, repeat = 40000)

My AI performance: 

Test #1
Over 10000 games... 
won 4386 times, 
lost 2241 times, 
and tied 3373 times. 
Winning rate = 43.86%

Test #2
Over 20000 games... 
won 8885 times, 
lost 4431 times, 
and tied 6684 times. 
Winning rate = 44.42%

Test #3
Over 30000 games... 
won 13335 times, 
lost 6660 times, 
and tied 10005 times. 
Winning rate = 44.45%

Test #4
Over 40000 games... 
won 17760 times, 
lost 9085 times, 
and tied 13155 times. 
Winning rate = 44.4%
